In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
from keras import layers
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import pandas as pd
import scipy
from tqdm import tqdm
import tensorflow as tf
from collections import Counter
import json
import itertools
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten,BatchNormalization
import os
from PIL import Image
import cv2
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve,roc_auc_score, auc
import matplotlib.pyplot as plt
import seaborn as sns

Data Gathering and Preprocessing

In [5]:
# create data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Define the data generator for the validation and test sets
val_test_datagen = ImageDataGenerator(rescale=1./255)

# fit the model on data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Train',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')
    
validation_generator = val_test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Valid',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')

Found 6000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    
    for root, _, files in os.walk(DIR):
        for file in files:
            if file.endswith('.png'):
                PATH = os.path.join(root,file)

                img = read(PATH)

                img = cv2.resize(img, (RESIZE,RESIZE))

                IMG.append(np.array(img))
    return IMG

real_train = np.array(Dataset_loader('/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Train/Real',64))
fake_train = np.array(Dataset_loader('/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Train/Fake',64))
real_test = np.array(Dataset_loader('/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Valid/Real',64))
fake_test = np.array(Dataset_loader('/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Valid/Fake',64))

Split dataset

In [10]:
# Create labels
real_train_label = np.ones(len(real_train))
fake_train_label = np.zeros(len(fake_train))
real_test_label = np.ones(len(real_test))
fake_test_label = np.zeros(len(fake_test))

# Merge data 
X_train = np.concatenate((real_train, fake_train), axis = 0)
Y_train = np.concatenate((real_train_label, fake_train_label), axis = 0)
X_test = np.concatenate((real_test, fake_test), axis = 0)
Y_test = np.concatenate((real_test_label, fake_test_label), axis = 0)

# Shuffle train data
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

# Shuffle test data
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]
